<a href="https://colab.research.google.com/github/ZathanZ61/EstruturasDeDados1Teste/blob/main/ComputabilidadeFabricaDeposito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install pulp
from pulp import *

# 1. Criação do Modelo
# O problema é de MINIMIZAÇÃO (custo).
modelo = LpProblem("Problema_de_Transporte_Logistico", LpMinimize)

# 2. Dados do Problema
FABRICAS = [1, 2, 3]
DEPOSITOS = [1, 2, 3]

# Custo de Transporte por tonelada (R$/t): Custo[Fabrica][Deposito]
# As rotas sem custo no diagrama são consideradas inexistentes (Infinito para o solver)
custos = {
    1: {1: 4.00, 2: 2.00, 3: 6.00},
    2: {1: 8.00, 2: 2.00, 3: 4.00},
    3: {1: 4.00, 2: 4.00, 3: 4.00}
}

# Produção Mensal (Oferta)
producao = {1: 2000, 2: 2000, 3: 2000}

# Capacidade Mensal (Demanda Máxima)
capacidade = {1: 3000, 2: 3000, 3: 1500}


# 3. Variáveis de Decisão (Xij)
# X_ij é a quantidade de toneladas transportadas da Fábrica i para o Depósito j.
rotas = [(i, j) for i in FABRICAS for j in DEPOSITOS]

X = LpVariable.dicts("X", rotas, 0) # Variáveis X_ij com limite inferior 0


# 4. FUNÇÃO OBJETIVO (Minimizar Custo Total)
# Z = Custo[i][j] * X_ij
modelo += lpSum([custos[i][j] * X[(i, j)] for (i, j) in rotas]), "Custo_Total"


# 5. RESTRIÇÕES

# A) Restrições de OFERTA (Toda a produção DEVE sair da Fábrica)
for i in FABRICAS:
    # A soma de todas as rotas saindo de uma fábrica i deve ser IGUAL à sua produção.
    modelo += lpSum([X[(i, j)] for j in DEPOSITOS]) == producao[i], f"Oferta_Fabrica_{i}"

# B) Restrições de DEMANDA (O que chega no Depósito NÃO PODE exceder a capacidade)
for j in DEPOSITOS:
    # A soma de todas as rotas chegando ao depósito j deve ser MENOR OU IGUAL à sua capacidade.
    modelo += lpSum([X[(i, j)] for i in FABRICAS]) <= capacidade[j], f"Demanda_Deposito_{j}"


# 6. RESOLUÇÃO DO MODELO
modelo.solve()

# Verifica se a solução foi encontrada
if LpStatus[modelo.status] == "Optimal":
    print("--- SOLUÇÃO ÓTIMA ENCONTRADA ---")

    # 7. EXIBIÇÃO DOS RESULTADOS
    print(f"\nCusto Mínimo Total de Transporte: R$ {value(modelo.objective):.2f}\n")
    print("DISTRIBUIÇÃO ÓTIMA (Toneladas Transportadas):")

    for i in FABRICAS:
        for j in DEPOSITOS:
            if X[(i, j)].varValue > 0:
                print(f"Fábrica {i} -> Depósito {j}: {X[(i, j)].varValue:.0f} t")

    print("\nVERIFICAÇÃO DE CAPACIDADE DOS DEPÓSITOS:")
    for j in DEPOSITOS:
        chegando = lpSum([X[(i, j)].varValue for i in FABRICAS])
        print(f"Depósito {j}: Chegou {value(chegando):.0f} t / Capacidade {capacidade[j]:.0f} t")

else:
    print(f"O problema não encontrou uma solução ótima. Status: {LpStatus[modelo.status]}")